# Logistic Regression Sentiment Classifier (Local Approach) for Twitter Political Data   
This notebook explains in details various steps implemented for the local approach for a Logistic Regression classifier. The steps we've taken have been broken into stages (represented as `<stage number> - <stage name>`).

# 1 - Getting Started
In this stage, we:

a. Install and import all dependencies required for the challenge.

b. Initialize Spark Session.

c. Read and cache dataset.

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=7f6004f12f8a2eedcea8c1ed830da067dc022c68a9aa8f4a221920789de1bbcc
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# %cd '/content/drive/My Drive/notebooks'


/content/drive/My Drive/notebooks


In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from src.utils.spark import initSparkSession
from src.utils.dataset import getPolarity
from src.jobs.spark_etl import extract
from src.transforms.Preprocessing import CleanTweet, polarityCalculator
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, IDF, StringIndexer, CountVectorizer
from pyspark.ml import Pipeline

In [5]:
# Spark Session for ETL job
sparkSession = initSparkSession(appName='Preprocessing')
sparkSession

In [6]:
df = extract(sparkSession)
df.cache()

DataFrame[created_at: timestamp, tweet_id: string, tweet: string, likes: float, retweet_count: float, source: string, user_id: string, user_name: string, user_screen_name: string, user_description: string, user_join_date: timestamp, user_followers_count: float, user_location: string, lat: double, long: double, city: string, country: string, continent: string, state: string, state_code: string, collected_at: timestamp, candidate: string]

# 2 - Spark User Defined Function (UDF)
In this stage, we implement an UDF for labelling our dataset using a rule-based algorithm. 

In [8]:
polarity = sparkSession.udf.register('Polarity', lambda record: getPolarity(record))

# 3 - Pipelines
In this stage, we experiment with different pipeline configurations (stages in the pipeline) to analyze which gives us the best results.    

In [9]:
def pipeline_clean_data():
    cleanTweet = CleanTweet()
    regexTokenizer = RegexTokenizer(inputCol="cleaned_tweet", outputCol="cleaned_tweet_words", pattern="[^a-zA-Z0-9_#]")
    stopWordsRemover = StopWordsRemover(inputCol='cleaned_tweet_words', outputCol='cleaned_tweet_nostop')
    return Pipeline(stages=[cleanTweet, regexTokenizer, stopWordsRemover])

def pipeline_transform_features():
    polarityCalc = polarityCalculator()
    countVectorizer = CountVectorizer(inputCol='cleaned_tweet_nostop', outputCol='cv')
    idf = IDF(inputCol='cv', outputCol="features")
    sentimentStringIdx = StringIndexer(inputCol = "polarity", outputCol = "label")
    return Pipeline(stages=[polarityCalc, countVectorizer, idf, sentimentStringIdx])

# 4 - Data Cleaning

In [10]:
pipelineModel = pipeline_clean_data().fit(df)
df = pipelineModel.transform(df)

In [11]:
pipelineModel = pipeline_transform_features().fit(df)
df = pipelineModel.transform(df)

In [12]:
df.show(truncate=False)

+----------------------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+-------------+-------------------+----------------------+-------------------+--------------------+------------------------------+------------------+-------------------+----------+--------------------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+--------------------------------------------------------------------------------------------------------------------

# 5 - Train-Test Split

In [17]:
df_train, df_test = df.randomSplit([0.70, 0.30], seed=123456)

# 6 - Access underlying RDD
The DataFrame API is not flexible enough for local (divide-and-conquer) approach. We don't do repartitioning as our dataset is large enough. 

In [18]:
rdd_train = df_train.rdd.cache()
rdd_test = df_test.rdd.cache()

In [19]:
rdd_train.getNumPartitions()

2

In [20]:
rdd_test.getNumPartitions()

2

In [82]:
all_columns = df_train.columns
all_columns

['tweet_id',
 'created_at',
 'tweet',
 'likes',
 'retweet_count',
 'source',
 'user_id',
 'user_join_date',
 'user_followers_count',
 'user_location',
 'lat',
 'long',
 'city',
 'state',
 'state_code',
 'tweet',
 'candidate',
 'cleaned_tweet_nostop',
 'polarity',
 'cv',
 'features',
 'label']

# 7 - Building Machine Learning Models in Parallel

In [72]:
from sklearn.linear_model import LogisticRegression

In [89]:
def build_model(it_partition_data):
    df_partition_data = pd.DataFrame(it_partition_data, columns=all_columns)
    clf = LogisticRegression(random_state=0)
    X_train = df_partition_data[['features']]
    y_train = df_partition_data[['label']]
    model = clf.fit(X_train.values, y_train.values)
    return [model]

In [90]:
models = rdd_train.mapPartitions(build_model).collect()
models

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 26.0 failed 1 times, most recent failure: Lost task 0.0 in stage 26.0 (TID 46) (7314938dd8da executor driver): org.apache.spark.api.python.PythonException: TypeError: float() argument must be a string or a real number, not 'SparseVector'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/lib/python3.10/dist-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/rdd.py", line 840, in func
    return f(iterator)
  File "<ipython-input-89-b8e56ec4728d>", line 6, in build_model


In [ ]:
type(models)

pyspark.rdd.PipelinedRDD

# 8 - Results
As the error above explains, the scikit-learn library does not handle complex types required for our dataset i.e. a sparse vector representing tweet data, and only accepts Strings or Numbers. This is the main limitation in exploring this approach further.

In [ ]:
sparkSession.stop()